# A deep dive into robomimic datasets

This notebook will provide examples on how to work with robomimic datasets through various python code examples. This notebook assumes that you have installed `robomimic` and `robosuite` (which should be on the `offline_study` branch).

In [1]:
import os
import json
import h5py
import numpy as np

import robomimic
import robomimic.utils.file_utils as FileUtils

# the dataset registry can be found at robomimic/__init__.py
from robomimic import DATASET_REGISTRY

ROBOMIMIC WARNING(
    No private macro file found!
    It is recommended to use a private macro file
    To setup, run: python /proj/vondrick3/sruthi/robots/robomimic/robomimic/scripts/setup_macros.py
)


## Download dataset

First, let's try downloading a simple dataset - we'll use the Lift (PH) dataset. Note that there are utility scripts such as `scripts/download_datasets.py` to do this for us, but for the purposes of this example, we'll use the python API.

In [2]:
# set download folder and make it
download_folder = "/tmp/robomimic_ds_example"
os.makedirs(download_folder, exist_ok=True)

# download the dataset
task = "lift"
dataset_type = "ph"
hdf5_type = "low_dim"
FileUtils.download_url(
    url=DATASET_REGISTRY[task][dataset_type][hdf5_type]["url"], 
    download_dir=download_folder,
)

# enforce that the dataset exists
dataset_path = os.path.join(download_folder, "low_dim_v141.hdf5")
assert os.path.exists(dataset_path)

low_dim_v141.hdf5: 21.7MB [00:01, 16.8MB/s]                                                                                                                                                   


## Read quantities from dataset

Next, let's demonstrate how to read different quantities from the dataset. There are scripts such as `scripts/get_dataset_info.py` that can help you easily understand the contents of a dataset, but in this example, we'll break down how to do this directly.

First, let's take a look at the number of demonstrations in the file.

In [3]:
download_folder = "/proj/vondrick3/sruthi/robots/robomimic/test_dataset"
dataset_path = '/proj/vondrick3/sruthi/robots/robomimic/datasets/lift_real/ph/demo.hdf5'
# open file
f = h5py.File(dataset_path, "r")

# each demonstration is a group under "data"
demos = list(f["data"].keys())
num_demos = len(demos)

print("hdf5 file {} has {} demonstrations".format(dataset_path, num_demos))

hdf5 file /proj/vondrick3/sruthi/robots/robomimic/datasets/lift_real/ph/demo.hdf5 has 200 demonstrations


In [4]:
f["data"]['demo_0'].keys()

<KeysViewHDF5 ['actions', 'dones', 'interventions', 'next_obs', 'obs', 'policy_acting', 'rewards', 'states', 'user_acting']>

Next, let's list all of the demonstrations, along with the number of state-action pairs in each demonstration.

In [5]:
# each demonstration is named "demo_#" where # is a number.
# Let's put the demonstration list in increasing episode order
inds = np.argsort([int(elem[5:]) for elem in demos])
demos = [demos[i] for i in inds]

for ep in demos:
    num_actions = f["data/{}/actions".format(ep)].shape[0]
    print("{} has {} samples".format(ep, num_actions))

demo_0 has 66 samples
demo_1 has 57 samples
demo_2 has 58 samples
demo_3 has 64 samples
demo_4 has 67 samples
demo_5 has 58 samples
demo_6 has 60 samples
demo_7 has 57 samples
demo_8 has 56 samples
demo_9 has 54 samples
demo_10 has 54 samples
demo_11 has 55 samples
demo_12 has 57 samples
demo_13 has 53 samples
demo_14 has 47 samples
demo_15 has 50 samples
demo_16 has 56 samples
demo_17 has 58 samples
demo_18 has 58 samples
demo_19 has 57 samples
demo_20 has 57 samples
demo_21 has 75 samples
demo_22 has 66 samples
demo_23 has 56 samples
demo_24 has 61 samples
demo_25 has 64 samples
demo_26 has 54 samples
demo_27 has 51 samples
demo_28 has 59 samples
demo_29 has 48 samples
demo_30 has 54 samples
demo_31 has 61 samples
demo_32 has 55 samples
demo_33 has 54 samples
demo_34 has 49 samples
demo_35 has 57 samples
demo_36 has 51 samples
demo_37 has 54 samples
demo_38 has 56 samples
demo_39 has 60 samples
demo_40 has 56 samples
demo_41 has 47 samples
demo_42 has 57 samples
demo_43 has 45 sample

Now, let's dig into a single trajectory to take a look at some of the quantities in each demonstration.

In [88]:
# look at first demonstration
demo_key = demos[0]
demo_grp = f["data/{}".format(demo_key)]

# Each observation is a dictionary that maps modalities to numpy arrays, and
# each action is a numpy array. Let's print the observations and actions for the 
# first 5 timesteps of this trajectory.
for t in range(5):
    print("timestep {}".format(t))
    obs_t = dict()
    # each observation modality is stored as a subgroup
    for k in demo_grp["obs"]:
        obs_t[k] = demo_grp["obs/{}".format(k)][t] # numpy array
    act_t = demo_grp["actions"][t]
    
    # pretty-print observation and action using json
    obs_t_pp = { k : obs_t[k].tolist() for k in obs_t }
    print("obs")
    print(json.dumps(obs_t_pp, indent=4))
    print("action")
    print(act_t)

timestep 0
obs


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [89]:
# we can also grab multiple timesteps at once directly, or even the full trajectory at once
first_ten_actions = demo_grp["actions"][:10]
print("shape of first ten actions {}".format(first_ten_actions.shape))
all_actions = demo_grp["actions"][:]
print("shape of all actions {}".format(all_actions.shape))

shape of first ten actions (10, 7)
shape of all actions (66, 7)


In [90]:
# the trajectory also contains the next observations under "next_obs", 
# for convenient use in a batch (offline) RL pipeline. Let's verify
# that "next_obs" and "obs" are offset by 1.
for k in demo_grp["obs"]:
    # obs_{t+1} == next_obs_{t}
    assert(np.allclose(demo_grp["obs"][k][1:], demo_grp["next_obs"][k][:-1]))
print("success")

success


In [91]:
# we also have "done" and "reward" information stored in each trajectory.
# In this case, we have sparse rewards that indicate task completion at
# that timestep.
dones = demo_grp["dones"][:]
rewards = demo_grp["rewards"][:]
print("dones")
print(dones)
print("")
print("rewards")
print(rewards)

dones
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]

rewards
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


In [92]:
# each demonstration also contains metadata
num_samples = demo_grp.attrs["num_samples"] # number of samples in this trajectory
mujoco_xml_file = demo_grp.attrs["model_file"] # mujoco XML file for this demonstration
print(mujoco_xml_file)

KeyError: "Unable to synchronously open attribute (can't locate attribute: 'model_file')"

Finally, let's take a look at some global metadata present in the file. The hdf5 file stores environment metadata which is a convenient way to understand which simulation environment (task) the dataset was collected on. 

In [94]:
env_meta = json.loads(f["data"].attrs["env_args"])
# note: we could also have used the following function:
# env_meta = FileUtils.get_env_metadata_from_dataset(dataset_path=dataset_path)
print("==== Env Meta ====")
print(json.dumps(env_meta, indent=4))
print("")

==== Env Meta ====
{
    "type": 3,
    "env_name": "RealPandaEnv",
    "env_kwargs": {
        "world": "Real",
        "robot": "panda",
        "controlType": "EEPosture",
        "config": {
            "world": {
                "type": "Real",
                "robot": "panda",
                "controlType": "EEPosture"
            },
            "sim_params": {
                "time_step": 0.001,
                "control_freq": 2,
                "steps_per_action": 40,
                "MAX_STEPS": 500
            },
            "redis": {
                "host": "172.16.0.1",
                "port": 6379
            },
            "redis_ws": {
                "host": "localhost",
                "port": 6379
            },
            "policy_freq": 20,
            "control_freq": 1000,
            "panda": {
                "arm": {
                    "path": "robot/franka_panda/panda.urdf",
                    "pose": [
                        0,
                        0,
 

## Visualizing demonstration trajectories

Finally, let's play some of these demonstrations back in the simulation environment to easily visualize the data that was collected.

It turns out that the environment metadata stored in the hdf5 allows us to easily create a simulation environment that is consistent with the way the dataset was collected!

In [95]:
import robomimic.utils.env_utils as EnvUtils

# create simulation environment from environment metedata
env = EnvUtils.create_env_from_metadata(
    env_meta=env_meta, 
    render=False,            # no on-screen rendering
    render_offscreen=True,   # off-screen rendering to support rendering video frames
)

TypeError: __init__() missing 1 required positional argument: 'ig_config'

In [97]:
import robomimic.utils.obs_utils as ObsUtils

# We normally need to make sure robomimic knows which observations are images (for the
# data processing pipeline). This is usually inferred from your training config, but
# since we are just playing back demonstrations, we just need to initialize robomimic
# with a dummy spec.
dummy_spec = dict(
    obs=dict(
            low_dim=["robot0_eef_pos"],
            rgb=[],
        ),
)
ObsUtils.initialize_obs_utils_with_obs_specs(obs_modality_specs=dummy_spec)


============= Initialized Observation Utils with Obs Spec =============

using obs modality: low_dim with keys: ['robot0_eef_pos']
using obs modality: rgb with keys: []


In [ ]:
import imageio

# prepare to write playback trajectories to video
video_path = os.path.join(download_folder, "playback.mp4")
video_writer = imageio.get_writer(video_path, fps=20)

In [ ]:
def playback_trajectory(demo_key):
    """
    Simple helper function to playback the trajectory stored under the hdf5 group @demo_key and
    write frames rendered from the simulation to the active @video_writer.
    """
    
    # robosuite datasets store the ground-truth simulator states under the "states" key.
    # We will use the first one, alone with the model xml, to reset the environment to
    # the initial configuration before playing back actions.
    init_state = f["data/{}/states".format(demo_key)][0]
    model_xml = f["data/{}".format(demo_key)].attrs["model_file"]
    initial_state_dict = dict(states=init_state, model=model_xml)
    
    # reset to initial state
    env.reset_to(initial_state_dict)
    
    # playback actions one by one, and render frames
    actions = f["data/{}/actions".format(demo_key)][:]
    for t in range(actions.shape[0]):
        env.step(actions[t])
        video_img = env.render(mode="rgb_array", height=512, width=512, camera_name="agentview")
        video_writer.append_data(video_img)

In [ ]:
# playback the first 5 demos
for ep in demos[5:10]:
    print("Playing back demo key: {}".format(ep))
    playback_trajectory(ep)

# done writing video
video_writer.close()

In [ ]:
# view the trajectories!
from IPython.display import Video
Video(video_path, embed=True)